<a href="https://colab.research.google.com/github/Shubhamkumar-op/AniSearch/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d hernan4444/animeplanet-recommendation-database-2020

Dataset URL: https://www.kaggle.com/datasets/hernan4444/animeplanet-recommendation-database-2020
License(s): CC0-1.0
 98% 106M/108M [00:07<00:00, 23.4MB/s]
100% 108M/108M [00:07<00:00, 16.0MB/s]


In [3]:
from zipfile import ZipFile

dataset = '/content/animeplanet-recommendation-database-2020.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
df = pd.read_csv('/content/anime.csv')
df = df.reset_index()
df.head()

,index,Anime-PlanetID,Name,Alternative Name,Rating Score,Number Votes,Tags,Content Warning,Type,Episodes,Finished,Duration,StartYear,EndYear,Season,Studios,Synopsis,Url
0,0,10,The Prince of Tennis,Tennis no Ouji-sama,4.037,10889,"Comedy, Drama, Shounen, Sports, Tennis, Based ...",Unknown,TV,178,True,Unknown,2001,2005,Fall 2001,"Production I.G, Trans Arts","Meet Ryoma Echizen, the cocky prince of tennis...",https://www.anime-planet.com/anime/the-prince-...
1,1,100,Neon Genesis Evangelion,Shinseiki Evangelion,4.248,54463,"Drama, Mecha, Sci Fi, Conspiracy, Kaijuu, Lone...","Emotional Abuse, Explicit Violence, Mature The...",TV,26,True,Unknown,1995,1996,Fall 1995,"GAINAX, Tatsunoko Production","In the future, a devastating event known as Se...",https://www.anime-planet.com/anime/neon-genesi...
2,2,1000,Full Metal Panic! The Second Raid,Unknown,4.35,23948,"Action, Comedy, Mecha, Sci Fi, Shounen, Milita...","Explicit Violence, Nudity",TV,13,True,Unknown,2005,2005,Summer 2005,Kyoto Animation,Half a year has passed since Sousuke Sagara to...,https://www.anime-planet.com/anime/full-metal-...
3,3,10000,22/7: Shampoo no Nioi ga Shita,Unknown,2.8,131,"Idols, School Life, CG Animation",Unknown,Music,1,True,5,2018,2018,Unknown,Unknown,The idol group 22/7 perform the song Shampoo n...,https://www.anime-planet.com/anime/22-7-shampo...
4,4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance,Unknown,1.271,21,Family Friendly,Unknown,Web,1,True,2,2017,2017,Unknown,Toei Animation,No synopsis yet - check back soon!,https://www.anime-planet.com/anime/oshiri-tant...


In [7]:
df.columns

Index(['index', 'Anime-PlanetID', 'Name', 'Alternative Name', 'Rating Score',
       'Number Votes', 'Tags', 'Content Warning', 'Type', 'Episodes',
       'Finished', 'Duration', 'StartYear', 'EndYear', 'Season', 'Studios',
       'Synopsis', 'Url'],
      dtype='object')

In [8]:
features = ['Rating Score','Number Votes','Studios','Synopsis', 'Tags', 'Episodes']

In [19]:
def combined_features(row):
    return str(row["Rating Score"])+" "+ str(row["Number Votes"])+" "+ str(row["Studios"])+" "+ str(row["Synopsis"])+" "+ str(row["Tags"])+" "+ str(row["Episodes"])+" "

def get_title_from_index(index):
    return df[df["index"] == index]["Name"].values[0]
def get_index_from_title(title):
    return df[df["Name"] == title]["index"].values[0]

In [11]:
df["combined_feature"]=df.apply(combined_features,axis=1)
df["combined_feature"].head()

,combined_feature
0,"4.037 10889 Production I.G, Trans Arts Meet Ry..."
1,"4.248 54463 GAINAX, Tatsunoko Production In th..."
2,4.35 23948 Kyoto Animation Half a year has pas...
3,2.8 131 Unknown The idol group 22/7 perform th...
4,1.271 21 Toei Animation No synopsis yet - chec...


In [15]:
cv = CountVectorizer()
count_matrix=cv.fit_transform(df["combined_feature"])

In [21]:
anime_liked = "One Piece"
cosine_sim = cosine_similarity(count_matrix)

In [22]:
liked_movie_index = cosine_sim[get_index_from_title(anime_liked)]
similar_anime = list(enumerate(liked_movie_index))
similar_anime.sort(key = lambda row: row[1],reverse=True)
for i in range(10):
    print(get_title_from_index(similar_anime[i][0]))

One Piece
One Piece Recap
Princess Mononoke
One Piece Movie 10: Strong World
One Piece: Dream Soccer King!
One Piece Movie 9: Episode of Chopper - The Miracle Winter Cherry Blossom
Sekai Meisaku Douwa: Aladdin to Mahou no Lamp
Nura: Rise of the Youkai Clan
Nodame Cantabile: Special Lesson
B-Legend! Battle B-Daman


In [23]:
import pickle
with open('model.pkl','wb') as f:
    pickle.dump(cosine_sim,f)